Importando as bibliotecas:

In [10]:
import os
import pathlib
import time
import codecs
import string
import re
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import urllib
import requests
from bs4 import BeautifulSoup

import nltk
import networkx as nx
import gensim
from wordcloud import WordCloud
#import textract

from IPython.core.display import Image
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Lendo o arquivo:

In [11]:
df = pd.read_csv('./mediaframe_export.csv')

Examinando as primeiras linhas:

In [12]:
df.head()

,id,website,url,publication_date,title,text
0,1202477,Estadão,"http://educacao.estadao.com.br/noticias/geral,...",2016-07-12T12:31:00,"Em um dia, MEC nomeia e exonera apoiador do 'E...",SÃO PAULO - Em um dia o Ministério da Educação...
1,1302701,Estadão,"http://cultura.estadao.com.br/noticias/geral,o...",2015-06-06T05:06:00,O movimento micou,"Nas varadas, a bateção de lata em cada apariçã..."
2,1178003,Estadão,http://politica.estadao.com.br/noticias/eleico...,2016-10-03T20:54:00,"No Twitter, Doria, Freixo e Haddad lideraram m...","Eleito prefeito de São Paulo no 1º turno, João..."
3,4211945,Estadão,"http://politica.estadao.com.br/noticias/geral,...",2016-11-09T09:06:00,Conselho deve livrar Bolsonaro de cassação,BRASÍLIA - O Conselho de Ética da Câmara deve ...
4,1275738,Estadão,http://brasil.estadao.com.br/blogs/direito-e-s...,2015-09-24T18:37:00,Do ódio social ao ódio político,Jovens negros e pobres sendo caçados em ônibus...


Examinando o tipo de dado das entradas:

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47450 entries, 0 to 47449
Data columns (total 6 columns):
id                  47450 non-null int64
website             47450 non-null object
url                 47450 non-null object
publication_date    47450 non-null object
title               47450 non-null object
text                47366 non-null object
dtypes: int64(1), object(5)
memory usage: 2.2+ MB


Transformando o campo "publication_date" em um camo do tipo datetime:

In [14]:
df['publication_date'] = pd.to_datetime(df['publication_date'])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47450 entries, 0 to 47449
Data columns (total 6 columns):
id                  47450 non-null int64
website             47450 non-null object
url                 47450 non-null object
publication_date    47450 non-null datetime64[ns]
title               47450 non-null object
text                47366 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 2.2+ MB


ordenando as colunas por data:

In [16]:
df.sort_values('publication_date', inplace=True)

In [17]:
df.head()

,id,website,url,publication_date,title,text
176,3119432,Estadão,"http://politica.estadao.com.br/noticias/geral,...",2001-02-22 03:00:00,"Militares aliciaram índias, conclui comissão","Militares aliciaram índias, conclui comissão\n..."
5585,3113146,Estadão,"http://politica.estadao.com.br/noticias/geral,...",2001-03-28 20:54:00,Militares protestam contra mudanças salariais,Militares protestam contra mudanças salariais\...
1065,3084322,Estadão,"http://politica.estadao.com.br/noticias/geral,...",2001-09-23 21:02:00,Câmara vai apurar denúncia contra presidente d...,Câmara vai apurar denúncia contra presidente d...
5315,3078276,Estadão,"http://politica.estadao.com.br/noticias/geral,...",2001-10-31 23:12:00,Comissão aprova acordo sobre a Base de Alcântara,Comissão aprova acordo sobre a Base de Alcânta...
409,3036199,Estadão,"http://politica.estadao.com.br/noticias/geral,...",2002-06-07 03:36:00,Meninos de rua protestam contra redução da ida...,Meninos de rua protestam contra redução da ida...


In [ ]:
df['text'] = df['text'].map(lambda x: str(x) + ' ')
texts = df['text'].sum()
texts[0:1000]

In [20]:
stop_pt = nltk.corpus.stopwords.words('portuguese')
stop_en = nltk.corpus.stopwords.words('english')
stopwords = stop_en + stop_pt
stopwords.extend(['-',''])

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\rsouza/nltk_data'
    - 'C:\\ProgramData\\Anaconda3\\nltk_data'
    - 'C:\\ProgramData\\Anaconda3\\share\\nltk_data'
    - 'C:\\ProgramData\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\rsouza\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
list_words = texts.split()
list_words = [l.strip().lower() for l in list_words]
list_words = [l.strip(string.punctuation) for l in list_words]
list_words = [l for l in list_words if l not in stopwords]

freqdist = Counter(list_words)
freqdist.most_common(30)

Exportando para um arquivo de texto:

In [11]:
import codecs
with codecs.open('./mediaframe_export.txt', 'w', "utf-8") as f:
    for entry in df.index:
        f.write('\n\n---------{}----------\n'.format(df.loc[entry, 'publication_date']))
        f.write('{}\n\n'.format(df.loc[entry, 'url']))
        f.write(df.loc[entry, 'text'])
        

Exportando para o Excel (alguns campos excedem ao máximo de número de caracteres em uma célula)

In [ ]:
#df.to_excel('./mediaframe_export.xls')